# Red Wine data - Linear Regression

In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Data preprocessing

In [2]:
#df = pd.read_csv('./red_wine.csv')
df = pd.read_csv('./white_wine.csv')

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,4898.0,6.854788,0.843868,3.80000,6.300000,6.80000,7.3000,14.20000
volatile acidity,4898.0,0.278241,0.100795,0.08000,0.210000,0.26000,0.3200,1.10000
citric acid,4898.0,0.334192,0.121020,0.00000,0.270000,0.32000,0.3900,1.66000
residual sugar,4898.0,6.391415,5.072058,0.60000,1.700000,5.20000,9.9000,65.80000
chlorides,4898.0,0.045772,0.021848,0.00900,0.036000,0.04300,0.0500,0.34600
free sulfur dioxide,4898.0,35.308085,17.007137,2.00000,23.000000,34.00000,46.0000,289.00000
total sulfur dioxide,4898.0,138.360657,42.498065,9.00000,108.000000,134.00000,167.0000,440.00000
density,4898.0,0.994027,0.002991,0.98711,0.991723,0.99374,0.9961,1.03898
pH,4898.0,3.188267,0.151001,2.72000,3.090000,3.18000,3.2800,3.82000
sulphates,4898.0,0.489847,0.114126,0.22000,0.410000,0.47000,0.5500,1.08000


In [5]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent], axis=1, keys=["Total","Percent"] )
missing_data[missing_data["Total"]>0]

,Total,Percent


No missing data. Yay!

Mean shift and scale the data:

In [6]:
scaler = StandardScaler()
data = scaler.fit_transform(df)
x = data[:,:-1]
y = np.array([data[:,-1]]).T
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7)
(m,n)=x_train.shape

## Training the data with TF

In [7]:
learning_rate = 0.01
training_epochs = 500

In [8]:
# define placeholders, variables, constants
X = tf.placeholder(tf.float32, [None,n])
Y = tf.placeholder(tf.float32,[None, 1])
X_test = tf.placeholder(tf.float32, [None,n])
Y_test = tf.placeholder(tf.float32,[None, 1])
w = tf.Variable(np.zeros([n,1]), name='weights', dtype=tf.float32)
#b = tf.Variable([1.], name='bias', dtype=tf.float32)

# define model
y_model = tf.matmul(X,w) #+b
# y_ = X @ w + b

# define cost
cost = tf.reduce_mean(tf.square(Y-y_model))

# select trainer and validation method
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
#RMSE = tf.reduce_mean(tf.square(Y_test- tf.matmul(X_test,w)-b))
RMSE = tf.reduce_mean(tf.square(Y_test- tf.matmul(X_test,w)))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    costs = []
    err = []
    for epoch in range(training_epochs):
        curr_cost, _ = sess.run([cost, train_op], feed_dict={X:x_train, Y: y_train})
        costs.append(curr_cost)
        err.append(sess.run(RMSE,feed_dict={X_test:x_test, Y_test: y_test}))
        if epoch % 10 == 0:
            print(epoch, ": ", curr_cost, ", ", err[-1] )
    w_val = sess.run(w)
    #b_val = sess.run(b)
    sess.close()
        
        

Instructions for updating:
Colocations handled automatically by placer.
0 :  1.0014428 ,  0.97962064
10 :  0.8874061 ,  0.8739578
20 :  0.83469856 ,  0.82602066
30 :  0.8047617 ,  0.7995323
40 :  0.78521466 ,  0.7827116
50 :  0.771404 ,  0.7711139
60 :  0.76120466 ,  0.7627229
70 :  0.75345856 ,  0.7564608
80 :  0.7474566 ,  0.7516796
90 :  0.7427317 ,  0.7479618
100 :  0.73896265 ,  0.74502635
110 :  0.735924 ,  0.7426776
120 :  0.7334514 ,  0.74077696
130 :  0.7314237 ,  0.739224
140 :  0.72975045 ,  0.7379437
150 :  0.7283624 ,  0.7368806
160 :  0.7272045 ,  0.73599166
170 :  0.72623545 ,  0.73524386
180 :  0.72542083 ,  0.73461115
190 :  0.7247333 ,  0.73407274
200 :  0.7241518 ,  0.7336124
210 :  0.723658 ,  0.7332164
220 :  0.723237 ,  0.73287445
230 :  0.72287744 ,  0.7325773
240 :  0.7225688 ,  0.7323178
250 :  0.72230285 ,  0.7320899
260 :  0.72207326 ,  0.73188835
270 :  0.7218734 ,  0.7317093
280 :  0.7216999 ,  0.7315495
290 :  0.72154707 ,  0.731406
300 :  0.721413 ,  0.73

In [9]:
y_pred=np.matmul(x_test,w_val)#+b_val
y_pred

array([[-0.44126675],
       [ 0.36735279],
       [-0.72138078],
       ...,
       [ 1.03519625],
       [ 0.52504778],
       [-0.10662834]])

In [10]:
y_test

array([[ 0.13787014],
       [ 0.13787014],
       [-0.99137392],
       ...,
       [ 0.13787014],
       [ 0.13787014],
       [ 0.13787014]])